# Importing Libraries

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install tensorflow-gpu==1.15.0
# !pip install keras==2.2.5
# !pip install h5py==2.10.0

In [3]:
!pip install mrcnn
!pip install imgaug

In [4]:
import tensorflow
import keras
print("tensorflow version:", tensorflow.__version__)
print("keras version:",keras.__version__)

tensorflow version: 1.15.0
keras version: 2.2.5


Using TensorFlow backend.


In [5]:
#deploy model using flask:
!pip install flask-ngrok

In [6]:
import os
import json

import numpy as np
from IPython.display import Image, display, clear_output
import shutil

from collections import Counter
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils.data_utils import get_file
from keras.models import Sequential, load_model,Model
import h5py
import skimage
import keras.engine as KE
KE.Layer
import pickle as pk
import mrcnn.model as modellib

import os
import sys

from mrcnn.config import Config
from mrcnn import utils
#import mrcnn.model as modellib
from mrcnn import visualize

%matplotlib inline
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

# Import Mask RCNN
#sys.path.append(ROOT_DIR)  # To find local version of the library

import keras.engine as KE

# from mrcnn import utils
# from mrcnn import visualize
from mrcnn.visualize import display_images
# from keras.layers import Layer
# from mrcnn import model
# # import mrcnn.model as modellib
from mrcnn.model import log
import cv2

import imgaug,h5py,IPython
# KE.Layer
%matplotlib inline
import zipfile
from skimage import data,io
from flask import render_template, request, Flask, url_for,flash, redirect,send_file
from flask_ngrok import run_with_ngrok
from shutil import make_archive
from tqdm import tqdm
from werkzeug.utils import secure_filename
from skimage import io

import warnings

# Detecting Car

In [7]:
model1 = VGG16(weights = 'imagenet')
with open('/content/drive/MyDrive/car-damage-detection-using-CNN/static/models/vgg16_cat_list.pk', 'rb') as f:
    categ_count = pk.load(f)
CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'


In [8]:
def get_predictions(preds, top=5):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',CLASS_INDEX_PATH,cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results


In [9]:

def prepare_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x


In [10]:
  def is_a_car(image,categ_list):
      img= prepare_image(image)

      out = model1.predict(img)
      preds = get_predictions(out, top=5)
      print("Identifying...")
      for pred in preds[0]:
          if pred[0:2] in categ_list:
              print(pred[0:2])
              return True
      return False

In [11]:
is_a_car('/content/drive/MyDrive/car damage detection/img/1.jpg',categ_count)


Identifying...
('n03770679', 'minivan')


True

# Detect Damage in Car

In [12]:
model2 = load_model('/content/drive/MyDrive/car-damage-detection-using-CNN/static/models/ft_model.h5')


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/usr/local/lib/python3.7/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [13]:

def is_damaged(image_path,model2):
    # urllib.request.urlretrieve(image_path, 'save.jpg')
    img = load_img(image_path, target_size=(256,256))
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)/255
    pred = model2.predict(x)
    #print("Car is Damaged")
    print(pred)
    if(pred[0][0]<=0.5):
      print("Car is Damaged")
      return True
    else:
      print("Car is not Damaged")
      return False

In [14]:
is_damaged('/content/drive/MyDrive/car damage detection/img/1.jpg',model2)

[[0.]]
Car is Damaged


True

# Detecting Damage in Car using mask-rcnn

In [15]:
os.chdir('/content/drive/MyDrive/car-damage-detection-using-CNN')
#ROOT DIRECTORY:
# Root directory 
ROOT_DIR= os.path.abspath("")
sys.path.append(ROOT_DIR) 
print("ROOT DIR:",ROOT_DIR)
#directory to save logs and trained model
# To find local version of the library
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

COCO_MODEL_PATH = os.path.join(ROOT_DIR,'mask_rcnn_coco.h5')
#DOWnload coco trained weights

if not os.path.exists(COCO_MODEL_PATH):
  utils.download_trained_weights(COCO_MODEL_PATH)

#Directory of images to run detection on
# IMAGE_DIR= os.path.join(ROOT_DIR,'images')

ROOT DIR: /content/drive/MyDrive/car-damage-detection-using-CNN


In [16]:
import custom,custom_1

In [17]:
# Root directory of the project
ROOT_DIR = os.getcwd()
sys.path.append(ROOT_DIR)  # To find local version of the library
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
custom_WEIGHTS_PATH = "/content/drive/MyDrive/car-damage-detection-using-CNN/static/models/mask_rcnn_scratch_0003.h5"  # TODO: update this path for best performing iteration weights
config = custom.CustomConfig()

In [18]:
config = custom.CustomConfig()
# ROOT_DIR = 'C:/Users/Sourish/Mask_RCNN'
# CUSTOM_DIR = os.path.join(ROOT_DIR + "/custom/")
# print(CUSTOM_DIR)
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [19]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)


# Create model in inference mode
# import tensorflow as tf
with tensorflow.device(DEVICE):
  model= modellib.MaskRCNN(mode='inference', model_dir= MODEL_DIR, config= config)



Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [20]:
# load the last best model you trained
# weights_path = model.find_last()[1]
# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)    
model.keras_model._make_predict_function()

Loading weights  /content/drive/MyDrive/car-damage-detection-using-CNN/static/models/mask_rcnn_scratch_0003.h5


# Deploying model


In [21]:
STATIC_FOLDER= os.path.join(ROOT_DIR, "static")
TEMPLATE_FOLDER= os.path.join(ROOT_DIR, "templates")
app= Flask(__name__)
app= Flask(__name__,static_folder=STATIC_FOLDER,template_folder=TEMPLATE_FOLDER)
run_with_ngrok(app)

app.secret_key = "secret key"
UPLOAD_PATH= os.path.join(STATIC_FOLDER, "uploads")
PRED_PATH=  os.path.join(STATIC_FOLDER, "prediction")
app.config['UPLOAD_FOLDER'] = UPLOAD_PATH
app.config['PRED_FOLDER'] = PRED_PATH
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
Allowed_extensions= ['jpg','jpeg','png','zip']
# zipped_extensions=['zip']

In [22]:
print("STATIC FOLDER", STATIC_FOLDER)
print("STATIC FOLDER", TEMPLATE_FOLDER)
print("STATIC FOLDER", UPLOAD_PATH)
print("STATIC FOLDER", PRED_PATH)


STATIC FOLDER /content/drive/My Drive/car-damage-detection-using-CNN/static
STATIC FOLDER /content/drive/My Drive/car-damage-detection-using-CNN/templates
STATIC FOLDER /content/drive/My Drive/car-damage-detection-using-CNN/static/uploads
STATIC FOLDER /content/drive/My Drive/car-damage-detection-using-CNN/static/prediction


In [23]:
def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in Allowed_extensions
@app.route('/')
def upload_form():
	return render_template('base.html')


In [24]:
@app.route('/', methods=['POST'])
def upload_image():
	fileupload, zipupload=False,False
	is_car, damaged_car= False,False
	
	# if no file is uploaded
	if 'file' not in request.files:
		flash('No file part')
		return redirect(request.url)
	file = request.files['file']

	if file.filename == '':
		flash('No image selected for uploading')
		return redirect(request.url)
	
	f= file.filename
	print("FILENAME IS:",f)
	# name of file without extension
	name= f.split('.')[-2]
	print("NAME OF FOLDER:",name)
	#extension of file uploaded:
	ext= f.split('.')[-1].lower()	
	class_names=['BG','scratch']
	# if uploaded file is either an image or zipped file
	if ext in Allowed_extensions:
		filename = secure_filename(file.filename)
		#save the file in uploads folder
		file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
		file_path= UPLOAD_PATH+'/'+filename
		
		#if file is image:
		if ext in ['jpg','jpeg','png']:
			fileupload=True
			# print("YOU HAVE UPLOADED AN IMAGE")
			print('upload_image filename: ' + filename)
			# flash('Image successfully uploaded and displayed below')
			#is a car
			# class_names=['BG','scratch']
			print("FILE PATH IS:",file_path)
			filename = file_path.split('/')[-1]
			print("Filename is:",filename)
			
			# folder = file_path.split('/')[-2]
			is_car= is_a_car(file_path,categ_count)
			if is_car:
				print("Car Found")
				# if car is damaged
				damaged_car= is_damaged(file_path,model2)
				if damaged_car:
					print("CAR IS DAMAGED")
					print("PREDICTING DAMAGE......")
		 			
					image = io.imread(file_path)  
					io.imshow(image)
					plt.show()
			
					#rescaling image
					if image.ndim != 3:
						image = skimage.color.gray2rgb(image)
				# If has an alpha channel, remove it for consistency
					if image.shape[-1] == 4:
							image = image[..., :3]
					#print(img.shape)
					image, window, scale, padding, crop = utils.resize_image(image,	min_dim=config.IMAGE_MIN_DIM, min_scale=config.IMAGE_MIN_SCALE,	max_dim=config.IMAGE_MAX_DIM,	mode=config.IMAGE_RESIZE_MODE)
					# print(image.shape)
					
					#Run Damage detection
					print("Running damage detection")
					img_arr = np.array(image)
					results = model.detect([image], verbose=1)
					ax= get_ax(1)
					ax = plt.gca()
					r = results[0]
					visualize.display_instances(img_arr, r['rois'], r['masks'], r['class_ids'],class_names, r['scores'],ax=ax, title='Predictions')
					plt.savefig(PRED_PATH+'/'+filename,bbox_inches='tight', pad_inches=-0.5,orientation= 'landscape')
					plt.show()
		 
					return render_template('damage.html',fileupload= fileupload, is_car= is_car, damaged_car= damaged_car, filename=filename)
				return render_template('damage.html',fileupload= fileupload,is_car= is_car, damaged_car= damaged_car, filename=filename) 
			return render_template('damage.html',fileupload= fileupload,is_car= is_car, damaged_car= damaged_car,filename= filename)
	 
		#if zip file is uploaded:
		if ext== 'zip':
			car,dam=0,0
			zipupload=True
			print("Zip file is uploaded")
			zip_ref = zipfile.ZipFile(os.path.join(UPLOAD_PATH, filename), 'r')
			zip_ref.extractall(UPLOAD_PATH)
			zip_ref.close()
			print("EXtracted zipped file")
			i=0
			img_dir= os.path.join(UPLOAD_PATH +'/'+ name)
			print("image directory:",img_dir)
			prediction_dir = os.path.join(PRED_PATH+'/'+name)
			if not prediction_dir:
				os.mkdir(prediction_dir, mode=0o777)
			print("PREDICTION DIR:", prediction_dir)
	 		# image_paths=[]
			for image_path in tqdm(os.listdir(img_dir)):
					path= os.path.join(img_dir+'/'+image_path)
					print("IMAGE PATH:",path)
					img = io.imread(path)
					plt.imshow(img)
		 			# If  there is a car:
					if is_a_car(path, categ_count):
						car+=1
						print("Car Found...")
						print("Detecting Damage....")
						# If car is damaged:
						if is_damaged(path,model2):
							dam+=1
							print("Damage Detected.....")
							print("Locating Damage....")
			 				# Locating  damage in car:
							# If grayscale. Convert to RGB for consistency.
							if img.ndim != 3:
								img = skimage.color.gray2rgb(img)
							# If has an alpha channel, remove it for consistency
							if img.shape[-1] == 4:
									img = img[..., :3]
							#print(img.shape)
							img, window, scale, padding, crop = utils.resize_image(img,
                                                              min_dim=config.IMAGE_MIN_DIM,
                                                              min_scale=config.IMAGE_MIN_SCALE,
                                                              max_dim=config.IMAGE_MAX_DIM,
                                                              mode=config.IMAGE_RESIZE_MODE)
							print(img.shape)
							#RUN DAMAGE DETECTION:
							img_arr = np.array(img)
							results = model.detect([img], verbose=1)
							ax= get_ax(1)
							ax = plt.gca()
							r = results[0]
							visualize.display_instances(img_arr, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'],ax=ax, title='Predictions')
							loc= PRED_PATH+'/'+str(i)
							plt.savefig(loc ,bbox_inches='tight', pad_inches=-0.5,orientation= 'landscape') 
							i+=1
							plt.show()
							# plt.imshow(loc)
						else:
							print("NO Damage Detected!!!!!!")
					else:
						print("No Car Found!!!!!!")
			# zipping the predicted image folder
			print("NUMBER OF CARS FOUND=",car)
			# flash("NUMBER OF CARS FOUND=",car)
			print("NUMBER OF DAMAGED CARS:",dam)
			# flash("NUMBER OF DAMAGED CARS:",dam)

			make_archive(os.path.join(PRED_PATH, filename), 'zip', root_dir= PRED_PATH)
			print("FOLDER SUCCESFULLY ARCHIEVED.........")

			return render_template('damage.html',zipupload= zipupload, car= car, dam= dam,result= 'damage_pred.zip')

				# res= predict_damage(image_path)
				# print(res)
		else:
			flash('Allowed image types are -> png, jpg, jpeg, gif and zipped extensions')
			return redirect(request.url)
	 
#display original image	
@app.route('/display/<filename>')
def display_image(filename):
	print('display_image filename: ' + filename)
	return redirect(url_for('static', filename='uploads/' + filename), code=301)
#display predicted image
@app.route('/<imagename>/predict_image')
def predict_image(imagename):
	print('predict_image filename: ' + imagename)
	return redirect(url_for('static', filename='prediction/' + imagename), code=301)

#dowload file
@app.route('/download/<filename>')
def download_file(filename):
	path= PRED_PATH+'/'+filename
	return send_file(path, as_attachment= True)
 

In [ ]:

if __name__=="__main__":                                           
  
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://63ba-35-185-78-107.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
